   # Lab Session 7  -  Jupyter Notebook


Import pandas_datareader to get the close date for every ticker symbol. Import requests library as rq to query the yahoo finance website and get the ticket symbol. Import fuzzywuzzy for getting a partial match against yahoo's website.

In [3]:
import pandas as pd
import pandas_datareader as pdr

import requests as rq
import datetime as dt

import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Reading the excel file saved in Lab Session 2, which was extracted from the website Information is beautiful.

Original Data source: https://docs.google.com/spreadsheets/d/1Je-YUdnhjQJO_13r8iTeRxpU2pBKuV6RVRHoYCgiMfg/edit#gid=322165570

In [5]:
Lab7 = pd.read_excel(r"C:\Users\Tanushree\Desktop\UNIVERSITY\Quarter 4\Dashboard\Lab1_Session\Session1.xlsx",sheetname = 0)

Getting the first five rows.

In [6]:
Lab7.head()

,Entity,alternative name,story,YEAR,records lost,ORGANISATION,METHOD OF LEAK,interesting story,NO OF RECORDS STOLEN,DATA SENSITIVITY,UNUSED,UNUSED.1,Exclude,Unnamed: 13,1st source link,2nd source link,3rd source,source name
0,NaN,NaN,Elaboration if there's an interesting story or...,"years are encoded (0=2004, 8 = 2012, 9 = 2013,...","(use 3m, 4m, 5m or 10m to approximate unknown ...",NaN,NaN,NaN,"(use 3m, 4m, 5m or 10m to approximate unknown ...",1. Just email address/Online information 20 SS...,NaN,NaN,Show this item in the viz?,NaN,NaN,NaN,NaN,NaN
1,AOL,American Online,A former America Online software engineer stol...,0,92000000,web,inside job,NaN,92000000,1,NaN,NaN,NaN,NaN,http://money.cnn.com/2004/06/23/technology/aol...,http://www.msnbc.msn.com/id/8985989/#.UFcN8RgUwaA,NaN,CNN
2,Automatic Data Processing,"Business outsourcing, payrolls, benefits",NaN,1,125000,financial,poor security,NaN,130000,20,NaN,NaN,NaN,NaN,http://abcnews.go.com/Technology/story?id=2160...,NaN,NaN,ABC
3,Ameritrade Inc.,Computer backup tape containing personal infor...,online broker,1,200000,financial,lost / stolen device,NaN,200000,20,NaN,NaN,NaN,NaN,http://www.nbcnews.com/id/7561268/,NaN,NaN,NaN
4,Citigroup,NaN,Blame the messenger! A box of computer tapes c...,1,3900000,financial,lost / stolen device,y,3900000,300,NaN,NaN,NaN,NaN,http://www.nytimes.com/2005/06/07/business/07d...,NaN,NaN,NY Times


Drop the first row which has description text about the column values.


In [7]:
Lab7 = Lab7.drop(Lab7.index[[0]])

Drop unnecessary columns from the sheet.

In [8]:
drop1 = ['story','alternative name','NO OF RECORDS STOLEN','interesting story','UNUSED' , 'UNUSED.1', 'Exclude', 'Unnamed: 13', '1st source link', '2nd source link', '3rd source', 'source name']
Lab7 = Lab7[[col for col in Lab7.columns if col not in drop1]]

Rename columns that have inconsistency in their name.

In [9]:
Lab7 = Lab7.rename(columns = {'Entity':'ENTITY','records lost':'RECORDS LOST' })

In the Data Sensitivity column, manipulating values as per the column description given,

[1- Just email address/Online information, 20- SSN/Personal details, 300- Credit card information, 4000- Email password/Health records, 50000- Full bank account details]

In [10]:
Lab7.loc[Lab7['DATA SENSITIVITY'] == 1, 'DATA SENSITIVITY'] = 'Just email address/Online information'
Lab7.loc[Lab7['DATA SENSITIVITY'] == 300, 'DATA SENSITIVITY'] = 'Credit card information'
Lab7.loc[Lab7['DATA SENSITIVITY'] == 4000, 'DATA SENSITIVITY'] = 'Email password/Health records'
Lab7.loc[Lab7['DATA SENSITIVITY'] == 50000, 'DATA SENSITIVITY'] = 'Full bank account details'

#Encountered some string values in column with numeric values
Lab7.loc[Lab7['DATA SENSITIVITY'] == '20', 'DATA SENSITIVITY'] = 'SSN/Personal details'
Lab7.loc[Lab7['DATA SENSITIVITY'] == '300', 'DATA SENSITIVITY'] = 'Credit card information'

Approximating the UNKNOWN FIGURES (as given use 3m, 4m, 5m or 10m to approximate unknown figures)


In [11]:
Lab7.loc[Lab7['ENTITY'] == 'OVH', 'RECORDS LOST'] = 5000000
Lab7.loc[Lab7['ENTITY'] == 'UbiSoft', 'RECORDS LOST'] = 10000000

Replacing year values from 1-14 with actual year values.

In [12]:
Lab7['YEAR'] = Lab7['YEAR'].replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],
[2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2017])

First searching the excel file with the yahoo API. Replacing {} brackets with the entity name and using the fuzz.partial_ratio to get the entities that match the yahoo Finance atleast 80% and return the ticker symbol.

In [13]:
def get_symbol(entity_name):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(entity_name)

    result = rq.get(url).json()

    for x in result['ResultSet']['Result']:
        if fuzz.partial_ratio(entity_name, x['name']) >= 80:
            if x['exchDisp'] == 'NYSE' or x['exchDisp'] == 'NASDAQ':
                return x['symbol'] 

Created a container list to get close dates


In [14]:
ticker = []

Iterate for each entity in the excelsheet and once we get the symbol then append it to the empty ticker[]

In [16]:
Entity_name = Lab7['ENTITY'].tolist()
for y in Entity_name:
    ticker.append(get_symbol(y)) 

There was an error while finding symbols for some companies using yahoo API. Added a try catch statement which prints 'No Ticker' when there is error.
The for loop will iterate through the empty data frame created(called Stock_price) and put the data in it. 

In [17]:
Stock_price = pd.DataFrame(columns = ['Symbol','Date', 'Close'])
for x in ticker:
    if x is not None:
        try:
            result = pdr.get_data_yahoo(x)
            result = result.reset_index()  
            result['Symbol'] = x
            result = result[['Symbol', 'Date', 'Close']]
            Stock_price = Stock_price.append(result)
        except:
            print('No Ticker')
    else:
        pass

No Ticker
No Ticker


Put the ticker symbols generated column after the last column of sheet Lab7

In [18]:
Lab7.insert(loc = 6,column = 'Ticker_Symbol', value = ticker)

Take the ticker column from Lab7 dataset and match with Ticker_Symbol column from Stock_price dataset and merge them

In [19]:
Lab7_Redesign = pd.merge(Lab7, Stock_price, left_on='Ticker_Symbol',right_on='Symbol', how='right')

In [21]:
Lab7_Redesign.tail()

,ENTITY,YEAR,RECORDS LOST,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY,Ticker_Symbol,Symbol,Date,Close
50986,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2017-10-30,109.010002
50987,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2017-10-31,108.529999
50988,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2017-11-01,109.800003
50989,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2017-11-02,108.919998
50990,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2017-11-03,108.989998


Download the data set for All Companies as a csv file

In [22]:
Lab7_Redesign.to_csv('Lab7_AllTicker.csv',index=False)

For showing the claim about Equifax, just extracted data about Equifax only

In [35]:
Lab7_Equifax = Lab7_Redesign.loc[Lab7_Redesign['ENTITY'] == 'Equifax']

In [36]:
Lab7_Equifax.head()


,ENTITY,YEAR,RECORDS LOST,ORGANISATION,METHOD OF LEAK,DATA SENSITIVITY,Ticker_Symbol,Symbol,Date,Close
49016,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2010-01-04,31.469999
49017,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2010-01-05,31.459999
49018,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2010-01-06,31.799999
49019,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2010-01-07,31.930000
49020,Equifax,2017,143000000,financial,hacked,Full bank account details,EFX,EFX,2010-01-08,31.940001


In [37]:
Lab7_Equifax.to_csv('Lab7_Equifax.csv',index=False)

Downloaded and added more Financial data from Nasdaq about Equifax for Fiscal years 2009-2017
This additional data would give insights about Revenue and EPS for 9 consecutive years. So, it clearly shows how in the year of the breach the income generated experienced a sudden fall as compared to the previous year. Also, Earnings per Share is the indicator of a company's profitability, it also dropped from 4.04 to 2.61.

Data source: http://www.nasdaq.com/symbol/efx/revenue-eps

In [44]:
Lab7_RevenueEPS0 = pd.read_excel(r"C:\Users\Tanushree\Desktop\UNIVERSITY\Quarter 4\Dashboard\Lab7_Session\Nasdaq data.xlsx",sheetname = 0)

In [45]:
Lab7_RevenueEPS0.head()

,Fiscal Year,Revenue,EPS,Dividends
0,2017,1689000000,2.61,0.78
1,2016,3144000000,4.04,1.32
2,2015,2663000000,3.55,1.16
3,2014,2436000000,2.97,1.00
4,2013,2303000000,2.84,0.88


Drop the column about Dividends

In [46]:
Lab7_RevenueEPS = Lab7_RevenueEPS0.drop('Dividends', axis=1)

Exporting the csv file 

In [48]:
Lab7_RevenueEPS.to_csv('Lab7_RevenueEPS.csv',index=False)
